<a href="https://colab.research.google.com/github/julmiha25-sys/Python/blob/main/%D0%9A%D0%B5%D0%B9%D1%811.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import os, re, glob, sys, csv # Работа с ОС, регул. выраж., поиск файлов, сист. пар-ми, csv-файлами

def process_files(src_folder, dest_folder):
    current_dir = os.getcwd() # Преобразование относ. путей в абсолютные
    if not os.path.isabs(src_folder):
        src_folder = os.path.join(current_dir, src_folder)
    if not os.path.isabs(dest_folder):
        dest_folder = os.path.join(current_dir, dest_folder)
    os.makedirs(dest_folder, exist_ok=True) # Если папки назначения нет, то создаем ее

    sys.path.append(os.path.join(os.getcwd(), '..', src_folder))
    if not os.path.exists(src_folder) or not os.path.exists(dest_folder):
        print(f"❌ Папки '{src_folder}' или '{dest_folder}' не существует")
        return None

    filter_files = r'\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d+\.csv' # Шаблон регулярных выражений
    files_cvs = glob.glob(os.path.join(src_folder, '*.csv')) # Поиск файлов по шаблону
    files = [f for f in files_cvs if re.match(filter_files, os.path.basename(f))] # Фильтрация файлов csv

    if not files_cvs:
        print(f"❌ В папке '{src_folder}' не найдено CSV файлов")
        return None

    if len(files_cvs) > len(files):
        print(f"❌ В папке '{src_folder}' присутствуют CSV файлы, но не в нужном формате.")
        print(f"    Ожидаемый формат: ГГГГ-ММ-ДД-ЧЧ-ММ-idмагазина.csv \n")
        if not files:
            return None

    print(f"✅ Загруженные файлы в кол-ве {len(files)}:")
    for f in files:
        print(f" - {os.path.basename(f)}")

    output_file = os.path.join(dest_folder, 'combined_data.csv')

    print(f"\n📊 Структура файлов:")
    all_headers_sets = [] # Список всех наборов заголовков
    headers_info = {} # Информация о заголовках каждого файла
    skipped_files_pre = [] # Пропущенные файлы

    # Функция для определения разделителя в CSV файле
    def detect_delimiter(file_path, encoding='utf-8', sample_lines=5):
        delimiters = [',', ';', '\t', '|'] # Определение разделителя в CSV по нескольким строкам
        delimiter_counts = {d: 0 for d in delimiters}

        try:
            with open(file_path, 'r', encoding=encoding) as f:
                lines_read = 0
                for line in f:
                    if lines_read >= sample_lines: # Чтение первых нескольких строк
                        break

                    for delimiter in delimiters:
                        delimiter_counts[delimiter] += line.count(delimiter) # Поиск разделителя с макс числом вхождений

                    lines_read += 1

            best_delimiter = max(delimiter_counts, key=delimiter_counts.get) # Разделитель с максимальным количеством вхождений

            if delimiter_counts[best_delimiter] == 0:  # , - по умолчанию
                return ','

            return best_delimiter
        except:
            return ','  # по умолчанию

    for idx, file_path in enumerate(files):
        file_name = os.path.basename(file_path)
        headers_found = False

        for encoding in ['utf-8', 'cp1251']: # По умолчанию работаем с 2-я кодировками
            try:
                # Определяем разделитель для файла
                delimiter = detect_delimiter(file_path, encoding)

                with open(file_path, 'r', encoding=encoding) as f:
                    reader = csv.reader(f, delimiter=delimiter, quotechar='"') # Создание объекта для построчного чтения csv-файла с определенным разделителем
                    current_headers = next(reader) # Получение строки заголовка

                    try: # Читаем первую строку данных для проверки файла на пустоту
                        first_data_row = next(reader) # Получение 1-й строки после заголовка
                        f.seek(0) # Возвращение к началу файла
                        next(reader) # Пропускаем заголовок снова

                    except StopIteration:
                        skipped_files_pre.append((idx + 1, file_name, "Файл содержит только заголовок"))
                        break

                    headers_found = True
                    headers_set = set(current_headers)
                    all_headers_sets.append(headers_set)
                    headers_info[file_name] = {
                        'path': file_path,
                        'headers': current_headers,
                        'headers_set': headers_set,
                        'encoding': encoding,
                        'delimiter': delimiter  # Сохраняем разделитель
                    }
                    print(f"  {idx + 1}. 📄 {file_name} - {len(current_headers)} колонок (разделитель: '{delimiter}')")
                    break

            except StopIteration: # Если файл пустой
                skipped_files_pre.append((idx + 1, file_name, "Файл пустой"))
                break
            except UnicodeDecodeError: # Если кодировка не utf-8 и не cp1251
                continue
            except Exception as e:  # Прочие исключения
                skipped_files_pre.append((idx + 1, file_name, f"Ошибка чтения: {str(e)[:50]}"))
                break

        if not headers_found and file_name not in [f[1] for f in skipped_files_pre]:
            skipped_files_pre.append((idx + 1, file_name, "Не удалось прочитать заголовки"))

    # Поиск общих колонок
    if not all_headers_sets:
        print("\n❌ Не удалось прочитать заголовки ни из одного файла")
        for num, fname, reason in skipped_files_pre:
            print(f"  {num}. ⭕ {fname} - {reason}")
        return None

    common_headers = set.intersection(*all_headers_sets) if len(all_headers_sets) > 1 else all_headers_sets[0] # Общие колонки

    if not common_headers:
        print("⚠️  Нет колонок, которые присутствуют во всех файлах")

        column_frequency = {}
        for headers_set in all_headers_sets:
            for header in headers_set:
                column_frequency[header] = column_frequency.get(header, 0) + 1 # Частота встречаемости колонок
        min_files = max(2, len(files) // 2)  # Минимум 2 файла или 50% от кол-ва колонок совпадают
        common_headers = {header for header, count in column_frequency.items()
                         if count >= min_files}
        print(f"   Выбраны колонки, имеющиеся хотя бы в {min_files} файлах")

    if not common_headers:
        print("\n❌ Не найдено общих колонок для обработки")
        print("   Статистика по колонкам:")
        column_frequency = {}
        for headers_set in all_headers_sets:
            for header in headers_set:
                column_frequency[header] = column_frequency.get(header, 0) + 1
        for header, count in sorted(column_frequency.items(), key=lambda x: (-x[1], x[0])):
            print(f"   - '{header}': {count} файл(ов)")
        return None

    target_headers = []
    for file_name, info in headers_info.items(): # Порядок колонок из первого файла, у которого есть все общие колонки
        if common_headers.issubset(info['headers_set']):
            target_headers = [h for h in info['headers'] if h in common_headers]
            break

    if not target_headers: # Или произвольный порядок
        target_headers = sorted(list(common_headers))

    print(f"\n🎯 Общие колонки ({len(target_headers)}): {', '.join(target_headers)}")

    # Поиск файлов для обработки >=50% общих колонок
    processable_files = []
    unprocessable_files = []

    for file_name, info in headers_info.items():
        common_with_file = common_headers.intersection(info['headers_set'])
        common_count = len(common_with_file)
        total_common = len(common_headers)

        if common_count >= total_common * 0.5:  # Есть 50% общих колонок
            processable_files.append(file_name)
            print(f"   ✅ {file_name}: {common_count}/{total_common} общих колонок")
        else:
            unprocessable_files.append((file_name, f"Совпадает только {common_count} из {total_common} колонок"))
            print(f"   ❌ {file_name}: только {common_count}/{total_common} общих колонок")

    if not processable_files:
        print("\n❌ Нет файлов с подходящей структурой для обработки")
        return None

    # ВСЕГДА ИСПОЛЬЗУЕМ ЗАПЯТУЮ КАК РАЗДЕЛИТЕЛЬ ДЛЯ ВЫХОДНОГО ФАЙЛА
    output_delimiter = ','  # Фиксированный разделитель для совместимости с эталоном
    print(f"\n🔧 Используем разделитель для выходного файла: '{output_delimiter}' (фиксированный для совместимости)")

    # Обработка подходящих файлов
    all_rows = [] # Список всех строк csv-файлов
    total_rows = 0  # Счетчик кол-ва строк из успешно обработанных файлов
    processed_files = 0  # Счетчик кол-ва успешно обработанных файлов
    skipped_files = []  # Список с кортежами с инф-ей о необработанных файлах

    print(f"\n🔄 Обработка файлов ({len(processable_files)} из {len(files)}):")

    for idx, file_name in enumerate(processable_files, 1):
        info = headers_info[file_name]
        file_path = info['path']
        current_headers = info['headers']
        encoding = info['encoding']
        delimiter = info['delimiter']

        try:
            with open(file_path, 'r', encoding=encoding) as f:
                reader = csv.reader(f, delimiter=delimiter, quotechar='"') # Создание объекта для построчного чтения csv-файла
                next(reader) # Пропускаем заголовок (уже прочитан)

                file_rows = []
                for row in reader: # Чтение данных
                    file_rows.append(row)

                if len(file_rows) == 0:
                    print(f"  {idx}. ⭕ {file_name} - Файл содержит только заголовки")
                    skipped_files.append((idx, file_name, "Нет данных"))
                    continue

                aligned_rows = [] # Выравнивание строк под общие колонки

                header_to_idx = {header: i for i, header in enumerate(current_headers)} # Словарь с колонками и индексами

                for row in file_rows:
                    aligned_row = []
                    for target_header in target_headers:
                        if target_header in header_to_idx:
                            idx_in_file = header_to_idx[target_header]
                            if idx_in_file < len(row):
                                # Сохраняем значение как есть, без преобразования в '0'
                                aligned_row.append(row[idx_in_file])
                            else:
                                aligned_row.append('')  # Пустая строка вместо '0'
                        else:
                            aligned_row.append('')  # Пустая строка вместо '0'
                    aligned_rows.append(aligned_row)

                all_rows.extend(aligned_rows) # Добавляем выровненные строки
                total_rows += len(file_rows) # Наращивание общего кол-ва строк
                processed_files += 1 # Наращивание кол-ва обраб. файлов
                print(f"  {idx}. ☑️ {file_name} - {len(file_rows):,} строк")

        except Exception as e:  # Прочие исключения
            print(f"  {idx}. ⭕ {file_name} - Ошибка обработки: {str(e)[:50]}")
            skipped_files.append((idx, file_name, f"Ошибка: {str(e)[:50]}"))

    if skipped_files_pre or unprocessable_files:
        print(f"\n📋 Пропущенные файлы:")

        for num, fname, reason in skipped_files_pre:
            print(f"  {num}. ⭕ {fname} - {reason}")

        for fname, reason in unprocessable_files:
            for orig_idx, orig_file in enumerate(files, 1): # Поиск № файла
                if os.path.basename(orig_file) == fname:
                    print(f"  {orig_idx}. ⭕ {fname} - {reason}")
                    break

    if not all_rows: # Ни один файл не удалось прочитать
        print("\n❌ Не удалось прочитать данные из подходящих файлов")
        return None

    try:
        date_index = None
        for i, header in enumerate(target_headers): # Находим индекс колонки date для сортировки
            if header.lower() == 'date':
                date_index = i
                break

        if date_index is not None:
            print(f"\n🔃 Сортируем строки по дате (колонка '{target_headers[date_index]}')...")

            def parse_date(date_str):  # Преобразование даты в формат для сортировки
                try:
                    year, month, day = map(int, date_str.split('-'))
                    return (year, month, day)
                except:
                    return (9999, 12, 31)

            all_rows.sort(key=lambda row: parse_date(row[date_index] if len(row) > date_index else '')) # Сортировка строк по дате

            if all_rows: # Проверка сортировки
                print(f"   Первая дата после сортировки: {all_rows[0][date_index] if len(all_rows[0]) > date_index else 'N/A'}")
                print(f"   Последняя дата после сортировки: {all_rows[-1][date_index] if len(all_rows[-1]) > date_index else 'N/A'}")
        else:
            print(f"\n⚠️  Колонка 'date' не найдена в общих заголовках. Сортировка не выполнена.")

    except Exception as e:
        print(f"\n⚠️  Ошибка при сортировке: {e}")

    output_file = os.path.join(dest_folder, 'combined_data.csv')

    try:

        with open(output_file, 'w', encoding='utf-8', newline='') as f:
            writer = csv.writer(f, delimiter=output_delimiter, quotechar='"', quoting=csv.QUOTE_MINIMAL)
            writer.writerow(target_headers)
            writer.writerows(all_rows)

        print(f"\n✅ CSV файл создан: {output_file}")
        print(f" - Всего строк: {total_rows}")
        print(f" - Файлов обработано: {processed_files} из {len(processable_files)} подходящих")
        print(f" - Файлов пропущено: {len(files) - processed_files}")
        print(f" - Колонок сохранено: {len(target_headers)}")
        if unprocessable_files:
            print(f" - Файлов с неподходящей структурой: {len(unprocessable_files)}")

    except Exception as e:
        print(f"\n❌ Ошибка при сохранении файла: {e}")
        return None

process_files('Input', 'Output')

❌ В папке '/content/Input' присутствуют CSV файлы, но не в нужном формате.
    Ожидаемый формат: ГГГГ-ММ-ДД-ЧЧ-ММ-idмагазина.csv 

✅ Загруженные файлы в кол-ве 5:
 - 2025-02-05-12-40-8.csv
 - 2025-02-01-13-40-10.csv
 - 2025-01-02-12-30-5.csv
 - 2025-02-03-05-30-5.csv
 - 2025-05-02-15-30-9.csv

📊 Структура файлов:
  1. 📄 2025-02-05-12-40-8.csv - 4 колонок (разделитель: ',')
  2. 📄 2025-02-01-13-40-10.csv - 2 колонок (разделитель: ',')
  3. 📄 2025-01-02-12-30-5.csv - 2 колонок (разделитель: ',')
  4. 📄 2025-02-03-05-30-5.csv - 1 колонок (разделитель: ',')
  5. 📄 2025-05-02-15-30-9.csv - 3 колонок (разделитель: ',')
⚠️  Нет колонок, которые присутствуют во всех файлах
   Выбраны колонки, имеющиеся хотя бы в 2 файлах

🎯 Общие колонки (3):  T h e r a p y , expr, юя e x p r 
   ❌ 2025-02-05-12-40-8.csv: только 0/3 общих колонок
   ✅ 2025-02-01-13-40-10.csv: 2/3 общих колонок
   ❌ 2025-01-02-12-30-5.csv: только 1/3 общих колонок
   ❌ 2025-02-03-05-30-5.csv: только 1/3 общих колонок
   ✅ 2025-

# Новый раздел